In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from deepface import DeepFace

In [3]:
# Accepting Class details:
import datetime

year = str(input("Enter year: "))
stream = str(input("Enter stream (CSE/IT): "))
course = str(input("Enter course (BTech or MTech): "))
subjectCode = str(input("Enter Subject Code: "))
subjectName = str(input("Enter Subject Name: "))
subjectCredits = str(input("Enter Subject Credits: "))
teacher = str(input("Enter Teacher Name: "))
timestamp = datetime.datetime.now()

In [4]:
import firebase_admin
from firebase_admin import db, credentials, storage

cred_obj = firebase_admin.credentials.Certificate('secret key.json')
default_app = firebase_admin.initialize_app(cred_obj, {
	'databaseURL': "https://smartattend-6da73-default-rtdb.firebaseio.com/",
    'storageBucket': 'smartattend-6da73.appspot.com'
})

# Get a database reference to our posts
ref = db.reference('Attendance')

In [5]:
students = {}

In [6]:
# bucket = storage.bucket()
# blob = bucket.get_blob(f'user_photos/2020/BTech/CSE/07901012020/face_photo')
# arr = np.frombuffer(blob.download_as_string(), np.uint8)
# img = cv2.imdecode(arr, cv2.COLOR_BGR2BGR555)
# cv2.imshow('image', img)
# print(img)
# cv2.waitKey(0)

In [7]:
bucket = storage.bucket()
list_blobs = list(bucket.list_blobs(prefix = f'user_photos/{year}/{course}/{stream}/'))

for blob_class in list_blobs:
    blob_name = blob_class.name
    if blob_name.endswith('face_photo'):
        blob = bucket.get_blob(blob_name)
        arr = np.frombuffer(blob.download_as_string(), np.uint8)
        img = cv2.imdecode(arr, cv2.COLOR_BGR2BGR555)
        # cv2.imshow('image', img)
        students[blob_name] = img

In [8]:
print(students)

{'user_photos/2020/BTech/CSE/07301012020/face_photo': array([[[254, 254, 254],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [242, 244, 244],
        [243, 245, 245],
        [243, 245, 245]],

       [[254, 254, 254],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [242, 244, 244],
        [243, 245, 245],
        [243, 245, 245]],

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [242, 244, 244],
        [243, 245, 245],
        [243, 245, 245]],

       ...,

       [[ 93,  78, 162],
        [ 93,  78, 162],
        [ 94,  79, 163],
        ...,
        [ 41,  26,  94],
        [ 40,  26,  90],
        [ 37,  23,  87]],

       [[ 91,  80, 152],
        [ 91,  80, 152],
        [ 92,  81, 153],
        ...,
        [ 42,  31,  87],
        [ 41,  29,  81],
        [ 39,  27,  79]],

       [[ 49,  42,  99],
        [ 49,  42,  99],
        [ 50,  43, 100],
        ...,
        [ 22,  15,  

In [9]:
camera = cv2.VideoCapture(0)

frame_count = camera.get(cv2.CAP_PROP_FRAME_COUNT)
fps = camera.get(cv2.CAP_PROP_FPS)

output = cv2.VideoWriter('./Video.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (640, 480))

if not camera.isOpened():
    print("Error in opening camera")
    exit()

while True:
    frame_count += 1
    if int(frame_count//fps) >= 3:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    ret, frame = camera.read()
    cv2.imshow('Capturing Video', frame)
    output.write(frame)

camera.release()
output.release()
cv2.destroyAllWindows()

In [10]:
video = cv2.VideoCapture('./Video.avi')
fps = video.get(cv2.CAP_PROP_FPS)
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

number_of_frames_required = 15
seconds = frame_count/(fps*number_of_frames_required)
print('video started')

for extracted_frame in range(number_of_frames_required):
    frame_id = int(fps*seconds*extracted_frame)
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    cv2.waitKey(0)
    cv2.imwrite("./frames/" + str(extracted_frame) + ".png", frame)

video started


In [11]:
min_number_of_heads = 10000
max_number_of_heads = 0
cropped = 0

for frame_number in range(number_of_frames_required):
    frame = cv2.imread('./frames/' + str(frame_number) + '.png')
    grayscale_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml') 
    detected_faces = face_cascade.detectMultiScale(grayscale_image)
    min_number_of_heads = min(min_number_of_heads, len(detected_faces))
    max_number_of_heads = max(max_number_of_heads, len(detected_faces))
    for (column, row, width, height) in detected_faces:
        # cv2.rectangle(frame,(column, row),(column + width, row + height),(0, 255, 0),4)
        cv2.imwrite("./refined-images/" + str(cropped) + ".png", frame[row:row+height, column:column+width])
        cropped += 1
    # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.show()

In [12]:
# HEAD COUNT
if min_number_of_heads == max_number_of_heads:
    print('Number of Students in the class are: ' + str(min_number_of_heads))
else:
    print("Number of Students in the class are within the range: " + str(min_number_of_heads) + '-' + str(max_number_of_heads))

Number of Students in the class are: 1


In [13]:
present = []

for cropped_img in range(cropped):
    for key in students:
        rollnumber = key.removeprefix(f'user_photos/{year}/{course}/{stream}/')[0:11]
        student_img = students[key]
        if(rollnumber not in present):
            try:
                result = DeepFace.verify(img1_path = "./refined-images/" + str(cropped_img) + ".png", img2_path = student_img)
                if (result['verified'] == True):
                    present.append(rollnumber)
            except:
                result = False

print("Attendance is: ")
print(present)

Attendance is: 
['07301012020']


In [14]:
attendance_ref = ref.child(f'{course}/{stream}/{year}/{teacher}/{subjectCode} {subjectName} {subjectCredits}/')

# Updating the number of classes held
try:
    totalClassesHeld = int(ref.get()[course][stream][year][subject]['totalClassesHeld'])+1
    print(totalClassesHeld)
    attendance_ref.update({
        'totalClassesHeld': str(totalClassesHeld)
    })
except:
    attendance_ref.update({
        'totalClassesHeld': 1
    })

In [15]:
for student in present:
    attendance_ref.push().set({
        'enrollmentNumber': student,
        'timestamp': str(timestamp)
})